In [1]:
# Dependencies
import requests
import json
import os
import pandas as pd
import numpy as np
import gmaps

# Google developer API key
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

In [2]:
# Import each year of data
data_file_2017 = "2017.csv"

In [3]:
# country_df = pd.read_csv(country_path)
happiness_17_df = pd.read_csv(data_file_2017)

In [4]:
happiness_17_df.sort_values(by=['Country']).head()

,Country,Abbr,Abbr_three,Region,Happiness_Rank,Happiness_Score,Whisker_high,Whisker_low,Economy_GDP_per_Capita,Family,Health_Life_Expectancy,Freedom,Generosity,Trust_Government_Corruption,Dystopia_Residual
0,Afghanistan,AF,AFG,Southern Asia,141,3.794,3.873661,3.714338,0.401477,0.581543,0.180747,0.106180,0.311871,0.061158,2.150801
1,Albania,AL,ALB,Central and Eastern Europe,109,4.644,4.752464,4.535536,0.996193,0.803685,0.731160,0.381499,0.201313,0.039864,1.490442
2,Algeria,DZ,DZA,Middle East and Northern Africa,53,5.872,5.978286,5.765714,1.091864,1.146217,0.617585,0.233336,0.069437,0.146096,2.567604
3,Angola,AO,AGO,Sub-Saharan Africa,140,3.795,3.951642,3.638358,0.858428,1.104412,0.049869,0.000000,0.097926,0.069720,1.614482
4,Argentina,AR,ARG,Latin America and Caribbean,24,6.599,6.690085,6.507915,1.185295,1.440451,0.695137,0.494519,0.109457,0.059740,2.614005


In [5]:
happiness_17_df.insert(0, 'Country_ID', range(1, 1 + len(happiness_17_df)))
happiness_17_df.head()

,Country_ID,Country,Abbr,Abbr_three,Region,Happiness_Rank,Happiness_Score,Whisker_high,Whisker_low,Economy_GDP_per_Capita,Family,Health_Life_Expectancy,Freedom,Generosity,Trust_Government_Corruption,Dystopia_Residual
0,1,Afghanistan,AF,AFG,Southern Asia,141,3.794,3.873661,3.714338,0.401477,0.581543,0.180747,0.106180,0.311871,0.061158,2.150801
1,2,Albania,AL,ALB,Central and Eastern Europe,109,4.644,4.752464,4.535536,0.996193,0.803685,0.731160,0.381499,0.201313,0.039864,1.490442
2,3,Algeria,DZ,DZA,Middle East and Northern Africa,53,5.872,5.978286,5.765714,1.091864,1.146217,0.617585,0.233336,0.069437,0.146096,2.567604
3,4,Angola,AO,AGO,Sub-Saharan Africa,140,3.795,3.951642,3.638358,0.858428,1.104412,0.049869,0.000000,0.097926,0.069720,1.614482
4,5,Argentina,AR,ARG,Latin America and Caribbean,24,6.599,6.690085,6.507915,1.185295,1.440451,0.695137,0.494519,0.109457,0.059740,2.614005


In [6]:
happiness_17_df["Lat"] = ""
happiness_17_df["Lng"] = ""
happiness_17_df.head()

,Country_ID,Country,Abbr,Abbr_three,Region,Happiness_Rank,Happiness_Score,Whisker_high,Whisker_low,Economy_GDP_per_Capita,Family,Health_Life_Expectancy,Freedom,Generosity,Trust_Government_Corruption,Dystopia_Residual,Lat,Lng
0,1,Afghanistan,AF,AFG,Southern Asia,141,3.794,3.873661,3.714338,0.401477,0.581543,0.180747,0.106180,0.311871,0.061158,2.150801,,
1,2,Albania,AL,ALB,Central and Eastern Europe,109,4.644,4.752464,4.535536,0.996193,0.803685,0.731160,0.381499,0.201313,0.039864,1.490442,,
2,3,Algeria,DZ,DZA,Middle East and Northern Africa,53,5.872,5.978286,5.765714,1.091864,1.146217,0.617585,0.233336,0.069437,0.146096,2.567604,,
3,4,Angola,AO,AGO,Sub-Saharan Africa,140,3.795,3.951642,3.638358,0.858428,1.104412,0.049869,0.000000,0.097926,0.069720,1.614482,,
4,5,Argentina,AR,ARG,Latin America and Caribbean,24,6.599,6.690085,6.507915,1.185295,1.440451,0.695137,0.494519,0.109457,0.059740,2.614005,,


In [7]:
def google_api_request(url, params=None):
    params = params or {}
    params['key'] = gkey
    result = requests.get(url, params=params).json()
    if 'error_message' in result:
        raise Exception(result['error_message'])
    return result['results']

def geocode_results(address):
    return google_api_request('https://maps.googleapis.com/maps/api/geocode/json', params={
        'address': address,
    })

def place_results(lat, lng, radius=8000, place_type=None, keyword=None, rankby=None):
    params = {
        'location': f"{lat},{lng}",
        'radius': radius,
    }
    if place_type:
        params['type'] = place_type
    if keyword:
        params['keyword'] = keyword
    if rankby:
        params['rankby'] = rankby
        del params['radius']
    return google_api_request('https://maps.googleapis.com/maps/api/place/nearbysearch/json', params)

In [8]:
for index, row in happiness_17_df.iterrows():
    city = row['Country']
    results = geocode_results(f"{city}")

    happiness_17_df.loc[index, "Lat"] = results[0]["geometry"]["location"]["lat"]
    happiness_17_df.loc[index, "Lng"] = results[0]["geometry"]["location"]["lng"]

In [9]:
happiness_17_df.head()

,Country_ID,Country,Abbr,Abbr_three,Region,Happiness_Rank,Happiness_Score,Whisker_high,Whisker_low,Economy_GDP_per_Capita,Family,Health_Life_Expectancy,Freedom,Generosity,Trust_Government_Corruption,Dystopia_Residual,Lat,Lng
0,1,Afghanistan,AF,AFG,Southern Asia,141,3.794,3.873661,3.714338,0.401477,0.581543,0.180747,0.106180,0.311871,0.061158,2.150801,33.9391,67.71
1,2,Albania,AL,ALB,Central and Eastern Europe,109,4.644,4.752464,4.535536,0.996193,0.803685,0.731160,0.381499,0.201313,0.039864,1.490442,41.1533,20.1683
2,3,Algeria,DZ,DZA,Middle East and Northern Africa,53,5.872,5.978286,5.765714,1.091864,1.146217,0.617585,0.233336,0.069437,0.146096,2.567604,28.0339,1.65963
3,4,Angola,AO,AGO,Sub-Saharan Africa,140,3.795,3.951642,3.638358,0.858428,1.104412,0.049869,0.000000,0.097926,0.069720,1.614482,-11.2027,17.8739
4,5,Argentina,AR,ARG,Latin America and Caribbean,24,6.599,6.690085,6.507915,1.185295,1.440451,0.695137,0.494519,0.109457,0.059740,2.614005,-38.4161,-63.6167


In [10]:
happiness_17_df.columns

Index(['Country_ID', 'Country', 'Abbr', 'Abbr_three', 'Region',
       'Happiness_Rank', 'Happiness_Score', 'Whisker_high', 'Whisker_low',
       'Economy_GDP_per_Capita', 'Family', 'Health_Life_Expectancy', 'Freedom',
       'Generosity', 'Trust_Government_Corruption', 'Dystopia_Residual', 'Lat',
       'Lng'],
      dtype='object')

In [11]:
happiness_17_df.to_csv("data2017.csv", encoding='utf-8', index=False, header=True)